In [1]:
import numpy as np
import torch
import pandas as pd
from sklearn.metrics import roc_auc_score
np.random.seed(2020)
torch.manual_seed(2020)
import pdb
import scipy.sparse as sps
import numpy as np
import torch
torch.manual_seed(2020)
from torch import nn
import torch.nn.functional as F
from math import sqrt

def sigmoid(x):
    return 1./(1. + np.exp(-x))

from dataset import load_data, load_features
from matrix_factorization import MF, MF_N_IPS, MF_N_DR_JL, MF_N_MRDR_JL
from baselines import MF, MF_IPS, MF_ASIPS, MF_SNIPS, MF_DR, MF_DR_JL, MF_MRDR_JL, MF_BaseModel
from models import MLP, MLP_exp, MLP_weibull, MLP_lognormal, MF_IPS_DF, MF_DR_JL_DF

from utils import gini_index, ndcg_func, get_user_wise_ctr, rating_mat_to_sample, binarize, shuffle, minU,recall_func, precision_func
from utils import ndcg_func_both, ndcg_func_feature, recall_func_both, recall_func_feature, generate_total_sample
mse_func = lambda x,y: np.mean((x-y)**2)
acc_func = lambda x,y: np.sum(x == y) / len(x)


dataset_name = "coat"

if dataset_name == "coat":
    train_mat, test_mat = load_data("coat")        
    x_train, y_train = rating_mat_to_sample(train_mat)
    x_test, y_test = rating_mat_to_sample(test_mat)
    num_user = train_mat.shape[0]
    num_item = train_mat.shape[1]

elif dataset_name == "yahoo":
    x_train, y_train, x_test, y_test = load_data("yahoo")
    x_train, y_train = shuffle(x_train, y_train)
    num_user = x_train[:,0].max() + 1
    num_item = x_train[:,1].max() + 1

print("# user: {}, # item: {}".format(num_user, num_item))
# binarize
y_train = binarize(y_train)
y_test = binarize(y_test)
n_train = x_train.shape[0]

train_user_ind = x_train[:, 0].astype('int')
train_item_ind = x_train[:, 1].astype('int')
test_user_ind = x_test[:, 0].astype('int')
test_item_ind = x_test[:, 1].astype('int')

# recover the complete matrix and retrieve the features

device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')
mf = MF(num_user, num_item, batch_size=128)
mf.to(device)
mf.fit(x_train, y_train, 
    lr=0.05,
    lamb=1e-3,
    tol=1e-4)

x_train_x, u_emb_train, v_emb_train = mf.forward(x_train, True)
x_test_x, u_emb_test, v_emb_test = mf.forward(x_test, True)

# for the feature set of training dataset and test dataset
feature_train = torch.cat([u_emb_train, v_emb_train], axis = 1).detach().cpu()
feature_test = torch.cat([u_emb_test, v_emb_test], axis = 1).detach().cpu()

# the feature of user and item
user_W = mf.W(torch.LongTensor(np.arange(num_user) ).to(device) ).detach().cpu()
item_H = mf.H(torch.LongTensor(np.arange(num_item) ).to(device) ).detach().cpu()

# L = 5, 20%, 19.66%
# L = 6.7, 15%, 15.05%
# L = 3.2, 30%, 30.12%
# L = 9.7, 10%, 10.02%
# L = 3.92, 25%, 24.99%

L = 3.2
sigmaH = 0.1

num_feature = feature_train.shape[1]
identity_p = np.diag(np.ones(num_feature))
mean_p = np.zeros(num_feature)

W_d = np.random.multivariate_normal(mean_p, sigmaH**2*identity_p)
print(W_d)

y_train_mask = np.zeros_like(y_train)
e_train = np.zeros_like(y_train, dtype='float')
d_train = np.zeros_like(y_train, dtype='float') + 1e5

prod = 0
for i in range(n_train):

    ts_i = np.random.uniform(0, L)
    lambda_i = np.exp( np.dot(W_d, feature_train[i, :]) )
    d_i = np.random.exponential(lambda_i)
    e_i = L - ts_i
    if d_i <= e_i:
        y_train_mask[i] = y_train[i]
        d_train[i] = d_i
    else:
        if y_train[i] == 1:
            prod += 1
        y_train_mask[i] = 0
    
    e_train[i] = e_i

print('mask proportion (within the original y_train = 1 )')
print( prod/(sum(y_train) )*100 )

test_pred = mf.predict(x_test)
mse_mf = mse_func(y_test, test_pred)
auc_mf = roc_auc_score(y_test, test_pred)
print('prediction for base model')
print(auc_mf)

ips_idxs = np.arange(len(y_test))
np.random.shuffle(ips_idxs)
y_ips = y_test[ips_idxs[:int(0.05 * len(ips_idxs))]]

feature_test = feature_test.to(device)

===>Load from coat data set<===
[train] rating ratio: 0.080000
[test]  rating ratio: 0.053333
# user: 290, # item: 300
[MF] epoch:8, xent:37.39917492866516
[-0.04494046 -0.09064704  0.11771674 -0.20835941  0.08493842 -0.07913483
 -0.04008389  0.0490871 ]
mask proportion (within the original y_train = 1 )
30.12147984538929
prediction for base model
0.6076920935476612


In [9]:
# for our model
# for IPS
mf_ips_df_acc = []
for repeat in np.arange(10):
    mf_ips_df = MF_IPS_DF(num_user, num_item, num_feature, batch_size=128)
    mf_ips_df.to(device)
    mf_ips_df.fit(x_train, y_train_mask, e_train, d_train, feature_train, y_ips,lr=0.03, lamb=6e-3, lamb1=4e-2, tol=1e-5)
    test_pred = mf_ips_df.predict(x_test, feature_test)
    mse = mse_func(y_test, test_pred)
    auc = roc_auc_score(y_test, test_pred)
    ndcg_res = ndcg_func_both(mf_ips_df, x_test, y_test, feature_test)
    recall_res = recall_func_both(mf_ips_df, x_test, y_test, feature_test)

    print(auc)

    mf_ips_df_acc.append([ mse, auc, np.mean(ndcg_res["ndcg_5"]), np.mean(ndcg_res['ndcg_10']), np.mean(recall_res['recall_5']), np.mean(recall_res['recall_10'])   ])

mf_ips_df_acc = np.array(mf_ips_df_acc)
mf_ips_df_mean = mf_ips_df_acc.mean(0)
mf_ips_df_sd = mf_ips_df_acc.std(0)

print("[MF_IPS_DF] test auc:", mf_ips_df_mean[1], ' sd: ', mf_ips_df_sd[1])
print(mf_ips_df_mean)
print(mf_ips_df_sd)

[MF-IPS-DF] epoch:31, xent:494.46990489959717
0.6645021996521696
[MF-IPS-DF] epoch:24, xent:493.81718349456787
0.668424184497034
[MF-IPS-DF] epoch:25, xent:493.9463939666748
0.6703237962230476
[MF-IPS-DF] epoch:21, xent:495.16478061676025
0.6738992459550605
[MF-IPS-DF] epoch:24, xent:493.9459147453308
0.6714140527189618
[MF-IPS-DF] epoch:26, xent:492.3830952644348
0.6775970704298543
[MF-IPS-DF] epoch:22, xent:496.0171432495117
0.6704135956985954
[MF-IPS-DF] epoch:27, xent:495.1417384147644
0.6707126695170509
[MF-IPS-DF] epoch:23, xent:495.4557914733887
0.6664531391731412
[MF-IPS-DF] epoch:25, xent:494.92343282699585
0.6637020273609046
[MF_IPS_DF] test auc: 0.6697441981225819  sd:  0.004000028558769318
[0.24107981 0.6697442  0.59560733 0.67061579 0.41090725 0.6984237 ]
[0.00030051 0.00400003 0.01278751 0.00874313 0.01315186 0.01247728]


In [10]:
# for DR
mf_dr_df_acc = []
for repeat in np.arange(10):
    mf_dr_df = MF_DR_JL_DF(num_user, num_item, num_feature, batch_size=128)
    mf_dr_df.to(device)
    mf_dr_df.fit(x_train, y_train_mask, e_train, d_train, feature_train, user_W, item_H,  y_ips, lr=0.03, lamb=8e-2, lambv=8e-3, tol=1e-5)
    test_pred = mf_dr_df.predict(x_test)
    mse = mse_func(y_test, test_pred)
    auc = roc_auc_score(y_test, test_pred)
    ndcg_res = ndcg_func(mf_dr_df, x_test, y_test)
    recall_res = recall_func(mf_dr_df, x_test, y_test)

    print(auc)

    mf_dr_df_acc.append([ mse, auc, np.mean(ndcg_res["ndcg_5"]), np.mean(ndcg_res['ndcg_10']), np.mean(recall_res['recall_5']), np.mean(recall_res['recall_10'])   ])

mf_dr_df_acc = np.array(mf_dr_df_acc)
mf_dr_df_mean = mf_dr_df_acc.mean(0)
mf_dr_df_sd = mf_dr_df_acc.std(0)

print("[MF_DR_JL_DF] test auc:", mf_dr_df_mean[1], ' sd: ', mf_dr_df_sd[1])
print(mf_dr_df_mean)
print(mf_dr_df_sd)

[MF-DR-JL-DF] epoch:22, xent:51969998.1875
0.707414459474821
[MF-DR-JL-DF] epoch:19, xent:52012784.9375
0.7052571454863633
[MF-DR-JL-DF] epoch:20, xent:51970732.25
0.7255632331368378
[MF-DR-JL-DF] epoch:28, xent:51928217.875
0.7251264152358681
[MF-DR-JL-DF] epoch:17, xent:51908958.125
0.719032133712869
[MF-DR-JL-DF] epoch:23, xent:51968474.5625
0.7051455969451553
[MF-DR-JL-DF] epoch:21, xent:52013547.6875
0.7173828778982321
[MF-DR-JL-DF] epoch:25, xent:52010246.0625
0.7205295327179413
[MF-DR-JL-DF] epoch:19, xent:51888235.0
0.7174657176727689
[MF-DR-JL-DF] epoch:23, xent:51951313.4375
0.7111584886313285
[MF_DR_JL_DF] test auc: 0.7154075600912185  sd:  0.007317014014786245
[0.3359154  0.71540756 0.62117989 0.69299181 0.43259145 0.71570684]
[0.00071286 0.00731701 0.01952625 0.01504092 0.01770227 0.00868716]


In [11]:
# for weibull
mlp_weibull_acc = []
for repeat in np.arange(10):
    mlp_weibull = MLP_weibull(num_feature, batch_size=128)
    mlp_weibull.to(device)
    mlp_weibull.fit(x_train, y_train_mask, e_train, d_train, feature_train, lr=0.05, lamb=1e-4,tol=1e-5)
    test_pred = mlp_weibull.predict(feature_test)
    mse = mse_func(y_test, test_pred)
    auc = roc_auc_score(y_test, test_pred)
    ndcg_res = ndcg_func_feature(mlp_weibull, x_test, y_test, feature_test)
    recall_res = recall_func_feature(mlp_weibull, x_test, y_test, feature_test)

    print(auc)

    mlp_weibull_acc.append([ mse, auc, np.mean(ndcg_res["ndcg_5"]), np.mean(ndcg_res['ndcg_10']), np.mean(recall_res['recall_5']), np.mean(recall_res['recall_10'])   ])

mlp_weibull_acc = np.array(mlp_weibull_acc)
mlp_weibull_mean = mlp_weibull_acc.mean(0)
mlp_weibull_sd = mlp_weibull_acc.std(0)

print("[MLP_weibull] test auc:", mlp_weibull_mean[1], ' sd: ', mlp_weibull_sd[1])
print(mlp_weibull_mean)
print(mlp_weibull_sd)

[MLP_weibull] epoch:15, xent:38.741294622421265
0.6326638874260628
[MLP_weibull] epoch:15, xent:38.77899479866028
0.6326862164668072
[MLP_weibull] epoch:18, xent:38.77335125207901
0.6326749069526639
[MLP_weibull] epoch:20, xent:38.78953808546066
0.6326722970647848
[MLP_weibull] epoch:18, xent:38.7636421918869
0.6326749069526639
[MLP_weibull] epoch:16, xent:38.75656741857529
0.6326055032675796
[MLP_weibull] epoch:19, xent:38.73294806480408
0.6326250290954167
[MLP_weibull] epoch:23, xent:38.74446028470993
0.632649098061414
[MLP_weibull] epoch:17, xent:38.73636597394943
0.6326574110376219
[MLP_weibull] epoch:18, xent:38.81410789489746
0.6326519979368354
[MLP_weibull] test auc: 0.632656125426185  sd:  2.3516781851340475e-05
[0.23713219 0.63265613 0.52590935 0.60560121 0.35991752 0.64515667]
[3.73548281e-03 2.35167819e-05 1.49192114e-04 1.55065708e-04
 1.06100796e-04 2.69318954e-04]


In [16]:
# for DR
mf_dr_acc = []
for repeat in np.arange(10):
    mf_dr = MF_DR(num_user, num_item, batch_size=128)
    mf_dr.to(device)
    mf_dr.fit(x_train, y_train_mask, y_ips, lr=0.06, lamb=1e-4,tol=1e-5)
    test_pred = mf_dr.predict(x_test)
    mse = mse_func(y_test, test_pred)
    auc = roc_auc_score(y_test, test_pred)
    ndcg_res = ndcg_func(mf_dr, x_test, y_test)
    recall_res = recall_func(mf_dr, x_test, y_test)

    print(auc)

    mf_dr_acc.append([ mse, auc, np.mean(ndcg_res["ndcg_5"]), np.mean(ndcg_res['ndcg_10']), np.mean(recall_res['recall_5']), np.mean(recall_res['recall_10'])   ])

mf_dr_acc = np.array(mf_dr_acc)
mf_dr_mean = mf_dr_acc.mean(0)
mf_dr_sd = mf_dr_acc.std(0)

print("[MF_DR] test auc:", mf_dr_mean[1], ' sd: ', mf_dr_sd[1])
print(mf_dr_mean)
print(mf_dr_sd)

[MF-DR] epoch:17, xent:57457.13250732422
0.6451885460333957
[MF-DR] epoch:15, xent:57463.63366699219
0.6554988597689844
[MF-DR] epoch:18, xent:57430.813232421875
0.6569978053742811
[MF-DR] epoch:13, xent:57500.952392578125
0.6570091148884244
[MF-DR] epoch:13, xent:57452.893981933594
0.6646121049306388
[MF-DR] epoch:15, xent:57482.85369873047
0.6606816137845384
[MF-DR] epoch:14, xent:57441.53924560547
0.6607745064605359
[MF-DR] epoch:12, xent:57462.97314453125
0.6476397140645505
[MF-DR] epoch:14, xent:57460.96276855469
0.6573161150330316
[MF-DR] epoch:14, xent:57442.282653808594
0.6569295616393653
[MF_DR] test auc: 0.6562647941977746  sd:  0.005566182197916437
[0.23670382 0.65626479 0.56864265 0.6447231  0.39067238 0.68082537]
[0.00030012 0.00556618 0.01434243 0.0131921  0.01506279 0.01403565]
